In [1]:
from birdclef.utils import get_spark
from IPython.display import Image, display

spark = get_spark()
display(spark)

# read straight from the bucket
df_meta = spark.read.csv(
    "gs://dsgt-clef-birdclef-2024/data/raw/birdclef-2023/train_metadata.csv"
)
df_meta.printSchema()
df_meta.show(vertical=True, n=1, truncate=100)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 22:06:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/18 22:06:25 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)

-RECORD 0----------------
 _c0  | primary_label    
 _c1  | secondary_labels 
 _c2  | type             
 _c3  | latitude         
 _c4  | longitude        
 _c5  | scientific_name  
 _c6  | common_name      
 _c7  | author           
 _c8  | license          
 _c9  | rating           
 _c10 | url              
 _c11 | filename         
only showing top 1 row



In [2]:
df_meta.show(n=5)

+-------------+----------------+--------+--------+---------+------------------+--------------------+-------------+--------------------+------+--------------------+--------------------+
|          _c0|             _c1|     _c2|     _c3|      _c4|               _c5|                 _c6|          _c7|                 _c8|   _c9|                _c10|                _c11|
+-------------+----------------+--------+--------+---------+------------------+--------------------+-------------+--------------------+------+--------------------+--------------------+
|primary_label|secondary_labels|    type|latitude|longitude|   scientific_name|         common_name|       author|             license|rating|                 url|            filename|
|      abethr1|              []|['song']|  4.3906|  38.2788|Turdus tephronotus|African Bare-eyed...|Rolf A. de By|Creative Commons ...|   4.0|https://www.xeno-...|abethr1/XC128013.ogg|
|      abethr1|              []|['call']| -2.9524|  38.2921|Turdus tephrono

In [11]:
import os
from pathlib import Path

in_root = Path("data/raw/birdclef-2023/train_audio")
out_root = Path("data/encodec_lm")

os.makedirs(out_root, exist_ok=True)

def get_out_dir(species_code):
    out_dir = out_root / species_code
    os.makedirs(out_dir, exist_ok=True)
    return out_dir

def get_out_path(filename, use_lm):
    species, file = filename.split("/")
    return get_out_dir(species) / (file[:-4] + ("_lm" * use_lm) + ".wav")

In [8]:
from encodec import EncodecModel
import torchaudio

bandwidth = 3.0
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(bandwidth)

file = "eaywag1/XC639588.ogg"
wav, sr = torchaudio.load(in_root / file)

/home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [12]:
from encodec.compress import compress, decompress
from encodec.utils import convert_audio

wav = convert_audio(wav, sr, model.sample_rate, model.channels)

compressed = compress(model, wav, use_lm=False)
compressed_lm = compress(model, wav, use_lm=True)

In [16]:
print(wav.shape)
print(len(compressed))
print(len(compressed_lm))

torch.Size([1, 584388])
9202
5539


In [13]:
out, out_sr = decompress(compressed)
out_lm, out_lm_sr = decompress(compressed_lm)

torchaudio.save(get_out_path(file, use_lm=False), out, sample_rate=out_sr, encoding="PCM_S", bits_per_sample=16)
torchaudio.save(get_out_path(file, use_lm=True), out_lm, sample_rate=out_lm_sr, encoding="PCM_S", bits_per_sample=16)

/home/acheung/birdclef-2024/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
